In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import json

In [2]:
# read the url and use beautiful soup to parse HTML response
# response = requests.get('https://www.vogue.com/fashion-shows/fall-2018-ready-to-wear/versace#details')
response = requests.get('https://www.vogue.com/fashion-shows/spring-2018-ready-to-wear/balenciaga#details')
soup = bs(response.text, 'html.parser')

# retrieve the designer and season details from <title> tag
vogue_designer_season = soup.title.text

# the website uses json data structure and java script within multiple <script> tags for the layout
# we will extract all of them using beautiful soup
json_data = soup.findAll(lambda tag: tag.name == 'script')
  
# retrieve date published for the collection
for item in json_data:
    date = re.search(r'"datePublished":"(.+?)T', str(item))
    if date:
        datepublished = str(date.group(1))
        break
print(datepublished)
print(json_data)

In [3]:
# Method - 2) This is another way of doing it and it does not have the <script> tags
# What I'm using is Regex (Regular Expression). 
# .group(1) works here because I'm using parenthesis in my regex. If I do group(0), it would be the whole value
# including the <script> tags. 

my_json = []
for x in json_data:
    testing = re.search(r'(\{\s*react:.+?\}\s*\});\s*<\/script>', str(x))
    
    if testing:
        my_json.append(testing.group(1))
        break
print(my_json)

['{ react: {"context":{"dispatcher":{"stores":{"ApplicationStore":{"channel":"runway","config":{"env":"production","siteConfig":{"allowedDomains":"www.vogue.com"}},"contentType":"runway_review","meta":{"canonical":"https://www.vogue.com/fashion-shows/spring-2018-ready-to-wear/balenciaga","id":"","title":"","description":"","ogTitle":"","ogDescription":"","twitterDescription":"","socialImage":"https://www.vogue.com/wp-content/uploads/2016/02/09/voguerunway-share.jpg"},"pageName":"fashionShowReview","primary":{"template":""},"pubDate":"","query":{},"seasonSlug":"spring-2018-ready-to-wear","subChannel":"balenciaga","subPageName":"","tags":[],"template":"runway_review","termSlug":""},"MetaStore":{"canonical":"https://www.vogue.com/fashion-shows/spring-2018-ready-to-wear/balenciaga","id":"","title":"","description":"","ogDescription":"","ogTitle":"","socialImage":"https://www.vogue.com/wp-content/uploads/2016/02/09/voguerunway-share.jpg","twitterDescription":""},"NewsletterStore":{"personal

In [4]:
# Every section in the website has a specific Client ID tagged. Fetch the Client ID tagged to all the images under "Detail" section
details_id = re.search(r'("detail":\s*\{\s*"__ref":\s*")(.+?)=="\s*\},', str(my_json))
if details_id:
    id_det=details_id.group(2)
else:
    print("No Client ID for Detail retrieved")
print(details_id.group(2))

RmFzaGlvblNob3dHYWxsZXJ5OjU5Yzg2Y2Y3MGY1ZmYwMTFkNzkxZjY5MQ


In [5]:
# Fetch the number of edges available for the Client ID. 
# Based on the observed source code, if the 'slidecount'=n and n>100, the max number of edges that can be retrived from the page is 100. 
# If the 'slidecount'= n and n<100, the max number of edges that can be retrived from the page is n.
# We will use this variable to define our regex for url retrieval later
regex_edge_ct = r'"' + re.escape(details_id.group(2)) + r'==":\s*\{(.+?)"slidecount":\s*(.+?),"slidesV2'
slidecount = re.search(regex_edge_ct, str(my_json), re.IGNORECASE)

if int(slidecount.group(2)) >= 100:
    edgemax = 99
else:
    edgemax = int(slidecount.group(2))-1
print(edgemax)

99


In [6]:
# using regex and edgemax variable we will extract the required portion of the json data which contains image urls
regex = r'"client:'+ re.escape(details_id.group(2)) + r'==:slidesV2\{\\\\"first\\\\\s*":100\}:edges:0":.*' + r'"client:'+ re.escape(details_id.group(2)) + r'==:slidesV2\{\\\\"first\\\\\s*":100\}:edges:' + re.escape(str(edgemax)) + r'".+?"client:' 
regex_ref1 = re.search(regex, str(my_json), re.IGNORECASE)
print(regex_ref1.group(0))

"client:RmFzaGlvblNob3dHYWxsZXJ5OjU5Yzg2Y2Y3MGY1ZmYwMTFkNzkxZjY5MQ==:slidesV2{\\"first\\":100}:edges:0":{"__id":"client:RmFzaGlvblNob3dHYWxsZXJ5OjU5Yzg2Y2Y3MGY1ZmYwMTFkNzkxZjY5MQ==:slidesV2{\\"first\\":100}:edges:0","__typename":"GallerySlidesEdge","node":{"__ref":"U2xpZGU6NTlkMGUzN2E5MzIyNzE0ZTY4NTRiM2Vm"}},"U2xpZGU6NTlkMGUzN2E5MzIyNzE0ZTY4NTRiM2Vm":{"__id":"U2xpZGU6NTlkMGUzN2E5MzIyNzE0ZTY4NTRiM2Vm","__typename":"Slide","id":"U2xpZGU6NTlkMGUzN2E5MzIyNzE0ZTY4NTRiM2Vm","photosTout":{"__ref":"TWVkaWE6NTlkMGUzN2E5MzIyNzE0ZTY4NTRiM2Vm"}},"TWVkaWE6NTlkMGUzN2E5MzIyNzE0ZTY4NTRiM2Vm":{"__id":"TWVkaWE6NTlkMGUzN2E5MzIyNzE0ZTY4NTRiM2Vm","__typename":"Image","id":"TWVkaWE6NTlkMGUzN2E5MzIyNzE0ZTY4NTRiM2Vm","altText":"","resizedUrl":"https://assets.vogue.com/photos/59d0e37a9322714e6854b3ef/master/pass/_ARC0012.jpg","url":"https://assets.vogue.com/photos/59d0e37a9322714e6854b3ef/master/pass/_ARC0012.jpg"},"client:RmFzaGlvblNob3dHYWxsZXJ5OjU5Yzg2Y2Y3MGY1ZmYwMTFkNzkxZjY5MQ==:slidesV2{\\"first\\":100}:e

In [8]:
# fetch the URL of all images for the Client ID retrieved
pattern = re.compile(r'"resizedUrl":"(.+?)"')
images = pattern.findall(str(regex_ref1.group(0)))
print(images)

['https://assets.vogue.com/photos/59d0e37a9322714e6854b3ef/master/pass/_ARC0012.jpg', 'https://assets.vogue.com/photos/59d0e3786d11ba27dea8eac5/master/pass/_ARC0018.jpg', 'https://assets.vogue.com/photos/59d0e379e4243e55c9af8da7/master/pass/_ARC0030.jpg', 'https://assets.vogue.com/photos/59d0e377d6bc165e4c9f27e9/master/pass/_ARC0046.jpg', 'https://assets.vogue.com/photos/59d0e37da0b05a63aaeb6948/master/pass/_ARC0052.jpg', 'https://assets.vogue.com/photos/59d0e37c9792f056ee202a17/master/pass/_ARC0061.jpg', 'https://assets.vogue.com/photos/59d0e37ed6bc165e4c9f27eb/master/pass/_ARC0068.jpg', 'https://assets.vogue.com/photos/59d0e37fa0b05a63aaeb694a/master/pass/_ARC0069.jpg', 'https://assets.vogue.com/photos/59d0e382d6bc165e4c9f27ed/master/pass/_ARC0077.jpg', 'https://assets.vogue.com/photos/59d0e3815da7fc651d2e9a07/master/pass/_ARC0085.jpg', 'https://assets.vogue.com/photos/59d0e387d6bc165e4c9f27ef/master/pass/_ARC0091.jpg', 'https://assets.vogue.com/photos/59d0e3885ab3fb3706c9e8ac/master

In [9]:
# write image url, collection, date published to a csv file and save with the name of the collection

data = pd.DataFrame({'imageurl':images})
data['collection'] = vogue_designer_season
data['datepublished'] = datepublished
data.to_csv(vogue_designer_season, sep=',', encoding='utf-8')
#return ("Write to file complete", vogue_designer_season,".csv")